In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import sys

sys.path.append(os.path.dirname(os.getcwd()))

import torch

from bertviz import head_view, model_view
from configs.base import Config
from transformers import BertTokenizer
from models.networks import MMSERALayerNorm
from data.dataloader import build_train_test_dataset

opt = Config()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cpu")
network =torch.load("/home/kuhaku/Code/EmotionClassification/code/3m-ser-private/scripts/checkpoints/MMSERALayerNorm/20230607-044147/weights/best_acc/checkpoint_0.pt",map_location=torch.device('cpu'),)
# network = MMSERALayerNorm(num_classes=opt.num_classes, num_attention_head=opt.num_attention_head, dropout=opt.dropout)
train_ds, test_ds = build_train_test_dataset("/home/kuhaku/Code/EmotionClassification/code/3m-ser-private/scripts/data")

In [13]:
int2str = ["Anger", "Happiness", "Sadness", "Neutral"]
for batch in test_ds:
    # Prepare batch
    text, label, sprectrome = batch["text"], batch["label"], batch["sprectrome"]
    label = torch.tensor([int(label)])
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)

    # Move inputs to cpu or gpu
    sprectrome = sprectrome.to(device)
    label = label.to(device)
    input_ids = input_ids.to(device)
    with torch.no_grad():
        # Forward pass
        output, attention = network(input_ids, sprectrome, output_attentions=True)
        # Calculate accuracy
        _, preds = torch.max(output, 1)
        accuracy = torch.mean((preds == label).float())
    if "he did wriggle so beautifully" in batch["text"]:
        print("{:<10} {} ".format(int2str[batch["label"]], accuracy) + batch["text"])
    # if (int(accuracy) == 0):
    #     print(batch["label"]), print(preds)
    #     head_view([attention[0]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]'))
    #     head_view([attention[1]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]' + ''.join([' {}'.format(i) for i in range(sprectrome.shape[0])])))
    # break

Anger      1.0 There is no point in coming down here.
Neutral    0.0 My name is Sara.
Neutral    1.0 Everyone wants to be a working actor, but they all work at something else.
Neutral    1.0 Well you know sometimes-- just with online transactions, things happens. so um-
Sadness    1.0 there's people that thought it was the right thing to do so now we have to- we have to do what we have to do for our country
Sadness    1.0 He just put it in my pocket that's the kind of guys I had.
Neutral    1.0 Alright?  Great.  Have a nice evening.
Neutral    1.0 I understand that. Um- So I'm going to put down here that you looking for your baggage, but we actually can't, uh this bag is likely not going to show up so what we're going to do-
Happiness  1.0 I'll never forgive you for that.
Anger      1.0 Amanda listen. Listen.
Neutral    0.0 It's become a habit with you.
Neutral    1.0 Is this a Masters or a P.h.D.?  Or can you...? Nice
Neutral    1.0 That obviously that's not true though.  I mean, you'

KeyboardInterrupt: 

In [8]:
model_view([attention[1]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]' + ''.join([' {}'.format(i) for i in range(sprectrome.shape[0])])))

<IPython.core.display.Javascript object>

In [ ]:
model_view([attention[1]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]'))